# VIEWERTOOL-VERSION 4.0

06.05.2020
namd psf, dcd
python 3.6.7 
requirements (nanodesign_dietz, MDAnalysis, numpy, mrcfile, nglview, ipywidgets)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%gui qt

In [14]:
import linkerapp as linkerAPP
import linkerappUtil as Util
import fileDialog

# select  project
select working directoy and required files. each cell opens a seperate FileDialog Popup

In [27]:
wd = fileDialog.gui_folderDialog()

In [24]:
json = fileDialog.gui_fileDialog(wd=wd, filter="caDNAno (*.json)")

In [ ]:
psf = fileDialog.gui_fileDialog(wd=wd, filter="topology (*.psf)")

In [ ]:
pdb = fileDialog.gui_fileDialog(wd=wd, filter="coordinates (*.dcd *.pdb)")

In [ ]:
mrc = fileDialog.gui_fileDialog(wd=wd, filter="cryo-map (*.mrc)")

In [7]:
viewer = linkerAPP.Viewer(wd, files=[json, psf, pdb, mrc])

[nanodesign.converters.cadnano.reader] INFO - Reading caDNAno design file /Users/elija/Dropbox (DIETZ LAB)/elijafeigl/Projects/a-atomic/a-iii__cMDff/a-iii__enrgcMDff/a-iii-e__whopperv5/a-iii-e-whopperv5-003__HB/whopperv5.json
[nanodesign.converters.cadnano.reader] INFO - Number of bases in a virtual helix 128 
[nanodesign.converters.cadnano.reader] INFO - Lattice type square 
[nanodesign.converters.cadnano.reader] INFO - Number of virtual helices read 44 
[nanodesign.converters.cadnano.reader] INFO - Number of scaffold bases read 2873 
[nanodesign.converters.cadnano.reader] INFO - Maximum number of lattice rows 24 
[nanodesign.converters.cadnano.reader] INFO - Maximum number of lattice columns 19 
[nanodesign.converters.cadnano.convert_design] INFO - Distance between adjacent helices 2.3 
[nanodesign.converters.cadnano.convert_design] INFO - Helix radius 1 
[nanodesign.converters.cadnano.convert_design] INFO - Number of bases in design 6147 
[nanodesign.converters.cadnano.convert_desig

# select helices and bases
either use the widget or specify custom selection:
* selection -> tuple containing to lists ([helix-ids],[base_ids])
* context number of voxels arround a selection for map zoning

#  save selection
-> name-temp.pdb/.mrc
-> name-temp_chim.pdb

write a pdb and an mrc of the selected atoms/regions to disk. (note: mrc file is overwritten whenever a new selection is made). name-temp_chim.pdb is corrected for use in chimera

* last: if True only write the final frame

## option 1: selection with widget 
(correct depiction for squarelattice)

In [8]:
sliders = viewer.make_selection()

Box(children=(VBox(children=(VBox(children=(HBox(children=(ToggleButton(value=False, layout=Layout(height='30p…

In [14]:
selection, context = viewer.eval_selection(*sliders)
atoms_selection, atoms_scaffold, atoms_staple, color_dict = viewer.select(*selection)


In [44]:
a = atoms_scaffold + atoms_staple
x = a.universe.atoms - atoms_selection
x

<ResidueGroup with 6041 residues>

In [47]:
x.select_atoms("name P and resname THY").positions

array([[-55.347054,  79.40373 , -60.625683],
       [-62.24963 ,  94.369835, -49.916286],
       [-56.695553,  98.25502 , -49.24538 ],
       ...,
       [ 60.549534, -26.550613,  20.85673 ],
       [ 59.11184 , -25.574207,  42.32003 ],
       [ 44.80192 ,  -5.790708,  49.934975]], dtype=float32)

In [ ]:
selection, context = viewer.eval_selection(*sliders)




path = viewer.project.input / "{}".format(viewer.project.name)


Util.mrc(atoms_selection, path=path, context=context, cut_box=True)
copyfile(str(path) + "-tmp.mrc", str(path) +"-" + "positive" + ".mrc")
Util.mrc(x, path=path, context=context, cut_box=True)
copyfile(str(path) + "-tmp.mrc", str(path) +"-" + "negative" + ".mrc")

# set cut_box=True for minimum size mrc-box
viewer.writepdb(atoms_selection)


In [ ]:
from shutil import copyfile
selection, context = viewer.eval_selection(*sliders)
atoms_selection, atoms_scaffold, atoms_staple, color_dict = viewer.select(*selection)
path = viewer.project.input / "{}".format(viewer.project.name)
Util.mrc(atoms_selection, path=path, context=context, cut_box=True)
# set cut_box=True for minimum size mrc-box
for ts in atoms_selection.universe.trajectory:
    print(ts.frame,)
    viewer.writepdb(atoms_selection, single_frame=True, frame=ts.frame)
    #copyfile(str(path) + "-tmp.pdb", str(path) + "-" + str(ts.frame) + ".pdb")
    copyfile(str(path) + "-tmp_chim.pdb", str(path) +"-" + str(ts.frame) + "_chim.pdb")

In [ ]:
viewer.writepdb(atoms_selection, single_frame=True, frame=0)

In [ ]:
x = "pointer-2_chim.pdb"

In [ ]:
number=2
print("save ./png/pointer-{:03d}.png format png width 3840 height 2160 \
        supersample 3 transparentBackground true;".format(number))

In [ ]:
#selection, context = viewer.eval_selection(*sliders)
selection

In [ ]:
u =atoms_selection.universe

In [ ]:
t = u.trajectory[0]

In [ ]:
t.frame

In [ ]:
str(path)

In [ ]:
# bullet from t0brick
start = 68
end = start + 93
selection = ([34, 25, 35, 36, 37, 46], range(start, end))
atoms_selection, _, _, _ = viewer.select(*selection)

print("1")
selection = ([27, 26], range(start+17, end-14))
atoms_add, _, _, _ = viewer.select(*selection)
atoms_selection += atoms_add
print("2")
selection = ([31, 40], range(start+21, end-10))
atoms_add, _, _, _ = viewer.select(*selection)
atoms_selection += atoms_add
print("3")
selection = ([44, 45], range(start+13, end-18))
atoms_add, _, _, _ = viewer.select(*selection)
atoms_selection += atoms_add
print("4")
selection = ([32, 33], range(start-7, end+4))
atoms_add, _, _, _ = viewer.select(*selection)
atoms_selection += atoms_add
print("5")
selection = ([39, 38], range(start-4, end))
atoms_add, _, _, _ = viewer.select(*selection)
atoms_selection += atoms_add
print("6")

path = viewer.project.input / "{}".format(viewer.project.name)
Util.mrc(atoms_selection, path=path, context=context, cut_box=True)
# set cut_box=True for minimum size mrc-box
viewer.writepdb(atoms_selection, single_frame=True, frame=-1) 



In [ ]:
# bullet from t0brick
start = 65
end = start + 93
selection = ([22, 13, 23, 24, 25, 34], range(start, end))
atoms_selection, _, _, _ = viewer.select(*selection)

print("1")
selection = ([15, 14], range(start+17, end-14))
atoms_add, _, _, _ = viewer.select(*selection)
atoms_selection += atoms_add
print("2")
selection = ([19, 28], range(start+21, end-10))
atoms_add, _, _, _ = viewer.select(*selection)
atoms_selection += atoms_add
print("3")
selection = ([32, 33], range(start+13, end-18))
atoms_add, _, _, _ = viewer.select(*selection)
atoms_selection += atoms_add
print("4")
selection = ([20, 21], range(start-7, end+4))
atoms_add, _, _, _ = viewer.select(*selection)
atoms_selection += atoms_add
print("5")
selection = ([27, 26], range(start-4, end))
atoms_add, _, _, _ = viewer.select(*selection)
atoms_selection += atoms_add
print("6")

path = viewer.project.input / "{}".format(viewer.project.name)
Util.mrc(atoms_selection, path=path, context=context, cut_box=True)
# set cut_box=True for minimum size mrc-box
viewer.writepdb(atoms_selection, single_frame=True, frame=-1) 


In [ ]:
atoms_selection

## option 2: custom selection

In [ ]:
########  Action required  #########
#selection = range(0,126)
#selection = ([66, 75, 13, 21], range(15,145))
#hb126 selection = (range(0,126),[85, 86, 87])#[ 60, 61, 85, 86,  110, 111, 135, 136])
#hb126 selection = ([35], range(40,150))#([35, 42, 54, 66], range(40,150))
selection = (range(0,126), range(0,100))
context = 5
####################################

In [49]:
atoms_selection, atoms_scaffold, atoms_staple, color_dict = viewer.select(*selection)
path = viewer.project.input / "{}".format(viewer.project.name)
#Util.mrc(atoms_selection, path=path, context=context)
viewer.writepdb(atoms_selection, single_frame=True, frame=-1) 

/Users/elija/.pyenv/versions/a-v_3.6.7/lib/python3.6/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  "".format(attrname, default))
/Users/elija/.pyenv/versions/a-v_3.6.7/lib/python3.6/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  "".format(attrname, default))
/Users/elija/.pyenv/versions/a-v_3.6.7/lib/python3.6/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  "".format(attrname, default))
/Users/elija/.pyenv/versions/a-v_3.6.7/lib/python3.6/site-packages/MDAnalysis/coordinates/PDB.py:902: UserWarning: Found no information for attr: 'tempfactors' Using default value of '0.0'
  "".format(attrname, default))


#  show selection
* isovalue: isosurface threshold
* backbone: true -> only backbone atoms displayed (faster); false -> all
* color_dict: None -> no staple coloring (significantely faster)

use the play button to show a movie of of the fitting or move the cursor to a specific timestep 

In [50]:
#color_dict = None
Util.fit_widget(atoms_selection, atoms_scaffold, atoms_staple, path=path, isolevel=1.35, backbone=False, color_dict=None)


ValueError: you must provide file extension if using file-like object or text content